In [2]:
import time
import datetime
import zhinst.core
import numpy as np
import matplotlib.pyplot as plt

daq = zhinst.core.ziDAQServer('192.168.55.138', 8004, 6)

CoreError: Could not connect to a Zurich Instruments data server on host '192.168.55.138' and port 8004. Make sure that the server is running, host / port names are correct, API / DataServer versions match. Execution of connect took longer than 20 seconds.

In [ ]:
daq.setDouble('/dev3554/oscs/0/freq', 34.53000000) # frequency
daq.setInt('/dev3554/demods/1/harmonic', 2) # harmonic
daq.setDouble('/dev3554/demods/0/timeconstant', 1.62282188) #timeconstant
daq.setDouble('/dev3554/demods/0/rate', 100.00000000) # data points per second
daq.setDouble('/dev3554/sigouts/0/amplitudes/1', 0.15000000) # amplitude in V
daq.setInt('/dev3554/sigouts/0/on', 1) # turns on button "top left force"
# daq.setInt('/dev3554/sigouts/0/enables/1', 0) # turn off button force
daq.setInt('/dev3554/sigouts/0/enables/1', 1) # turns on button "bottom fight force"

## both buttons need to me turned on to start parametric resonance

In [ ]:
daq_module = daq.dataAcquisitionModule()
daq_module.set('triggernode', '/dev3554/demods/0/sample.R')
daq_module.set('preview', 1)
daq_module.set('device', 'dev3554')
daq_module.set('historylength', 100)
daq_module.set('hysteresis', 0.01000000)
daq_module.set('level', 0.10000000)
daq_module.set('save/directory', '\\home\mdudko\My Documents\Zurich Instruments\LabOne\WebServer')
daq_module.set('clearhistory', 1)
daq_module.set('bandwidth', 0.00000000)

In [ ]:
turnOn = True
ttime = 1
length = 1

In [ ]:
if turnOn:
    daq.setInt('/dev3554/sigouts/0/enables/1', 1)
    time.sleep(ttime)


daq_module.set('endless', 0)
# daq_module.subscribe('/dev3554/demods/0/sample.R.avg')
daq_module.subscribe('/dev3554/demods/0/sample.X.avg')
daq_module.subscribe('/dev3554/demods/0/sample.Y.avg')
daq_module.execute()

p = length*100
daq_module.set('grid/cols', p)
daq.setInt('/dev3554/sigouts/0/enables/1', 0)
daq_module.set('forcetrigger', 1)

while daq_module.progress() < 1.0 and not daq_module.finished():
    time.sleep(5)
    print(f"Progress {float(daq_module.progress()) * 100:.2f} %\r")

data = daq_module.read()

x = np.array(data['dev3554']['demods']['0']['sample.x.avg'][0]['value'])[0]
y = np.array(data['dev3554']['demods']['0']['sample.y.avg'][0]['value'])[0]

myString = datetime.datetime.now().strftime("%H-%M")
np.savetxt(f"{myString}.csv",np.column_stack((x,y)),delimiter=",",fmt='%s',comments='')

plt.scatter(x*1000,y*1000)
plt.show()

In [11]:
import numpy as np
import matplotlib.pyplot as plt

datar = np.genfromtxt(r"\DATA\11-51.csv", delimiter=",")
xr = datar[:,0]
yr = datar[:,1]
plt.scatter(xr*1000,yr*1000)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '\\DATA\\11-51.csv'